In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# mlp for multi-output regression
# from numpy import mean
# from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
aa = pd.read_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\AA-NACL.csv',header=0,
                    names=['temp','conductivity','velocity','nacl','aa'],dtype='float32')
ethanol = pd.read_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\ETHANOL-NACL.csv',header=0,
                    names=['temp','conductivity','velocity','nacl','aa'],dtype='float32')
khi = pd.read_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\KHI-NACL.csv',header=0,
                    names=['temp','conductivity','velocity','nacl','aa'],dtype='float32')
meg = pd.read_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\MEG-NACL.csv',header=0,
                    names=['temp','conductivity','velocity','nacl','aa'],dtype='float32')

In [4]:
aa.head()

,temp,conductivity,velocity,nacl,aa
0,0.0,0.090,1402.699951,0.0,0.2
1,0.0,0.190,1404.099976,0.0,0.5
2,0.0,0.329,1405.800049,0.0,1.0
3,0.0,0.430,1408.000000,0.0,1.5
4,0.0,0.550,1409.099976,0.0,2.0


In [3]:
aa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   temp          105 non-null    float32
 1   conductivity  105 non-null    float32
 2   velocity      105 non-null    float32
 3   nacl          105 non-null    float32
 4   aa            105 non-null    float32
dtypes: float32(5)
memory usage: 2.2 KB


In [8]:
X_train = aa.drop(aa.iloc[:,3:5],axis=1)
X_train

,temp,conductivity,velocity
0,0.0,0.090000,1402.699951
1,0.0,0.190000,1404.099976
2,0.0,0.329000,1405.800049
3,0.0,0.430000,1408.000000
4,0.0,0.550000,1409.099976
...,...,...,...
100,25.0,48.340000,1531.300049
101,25.0,48.169998,1530.599976
102,25.0,47.509998,1531.300049
103,25.0,46.930000,1532.099976


In [7]:
y_train = aa.iloc[:,3:5]
y_train.head()

,nacl,aa
0,0.0,0.2
1,0.0,0.5
2,0.0,1.0
3,0.0,1.5
4,0.0,2.0
...,...,...
100,3.0,1.0
101,3.0,1.5
102,3.0,2.0
103,3.0,2.8


In [32]:
# Additionally, it is good practice to use k-fold cross-validation instead of train/test splits 
# of a dataset to get an unbiased estimate of model performance when making predictions on new 
# data. Again, only if there is not too much data and the process can be completed in a reasonable time.

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size=0.2,random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_val, y_val,test_size=0.5,random_state=42)

In [33]:
from sklearn.impute import SimpleImputer
# print total missing
X_train.isnull().sum()
# define imputer
imputer = SimpleImputer(strategy='mean')
imputer.fit(X_train)
Xtrans = imputer.transform(X_train)
print(Xtrans[:3, :])

[[  15.     37.6  1506.  ]
 [   4.     29.36 1461.8 ]
 [  25.     48.34 1531.3 ]]


In [27]:
# FEATURE SELECTION WITH RFE - RECURSIVE FEATURE ELIMINATION
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor


# rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=3)
rfe = RFE(estimator=DecisionTreeRegressor(), n_features_to_select=3)

# fit RFE
rfe.fit(X_train, y_train)
# summarize all features
for i in range(X_train.shape[1]):
	print('Column: %d, Selected=%s, Rank: %d' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, Selected=True, Rank: 1
Column: 1, Selected=True, Rank: 1
Column: 2, Selected=True, Rank: 1


In [34]:
from sklearn.preprocessing import MinMaxScaler
trans = MinMaxScaler()
# transform the data
X_norm = trans.fit_transform(Xtrans)
# summarize data after the transform
print(X_norm[:3, :])

[[0.59999996 0.77234906 0.7964115 ]
 [0.16       0.6030491  0.45163822]
 [1.         0.99301434 0.9937601 ]]


In [74]:

# fit RFE
rfe.fit(X_val, y_val)
# summarize all features
for i in range(X_val.shape[1]):
	print('Column: %d, Selected=%s, Rank: %d' % (i, rfe.support_[i], rfe.ranking_[i]))

X_norm_val = trans.fit_transform(Xtrans)
# summarize data after the transform
print(X_norm_val[:3, :])

Column: 0, Selected=True, Rank: 1
Column: 1, Selected=True, Rank: 1
Column: 2, Selected=True, Rank: 1
[[0.59999996 0.77234906 0.7964115 ]
 [0.16       0.6030491  0.45163822]
 [1.         0.99301434 0.9937601 ]]


In [56]:
# define the model
# {'number of input'}
# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs))
	model.compile(loss='mae', optimizer='adam')
	return model

In [45]:
# df.reset_index().values.ravel().view(dtype=[('index':int),('A':float),])
y_train_array = y_train.to_numpy(dtype='float')
print(y_train_array[:3, :])

[[3.         2.79999995]
 [3.         0.5       ]
 [3.         1.        ]]


In [53]:
y_train_array.dtype

dtype('float64')

In [57]:
model = get_model(3, 2)
model

In [60]:
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
	results = list()
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	# define evaluation procedure
	cv = RepeatedKFold(n_splits=10, n_repeats=2, random_state=1)
	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]
		# define model
		model = get_model(n_inputs, n_outputs)
		# fit model
		model.fit(X_train, y_train, verbose=0, epochs=100)
		# evaluate model on test set
		mae = model.evaluate(X_test, y_test, verbose=0)
		# store result
		print('>%.3f' % mae)
		results.append(mae)
	return results

In [61]:
# evaluate model
results = evaluate_model(X_norm, y_train_array)
# summarize performance
print('MAE: %.3f (%.3f)' % (np.mean(results), np.std(results)))

>1.083
>0.442
>0.834
>0.812
>0.845
>1.189
>0.745
>0.785
>0.482
>0.542
>0.691
>0.851
>0.950
>0.513
>0.714
>0.939
>0.657
>0.878
>0.938
>0.793
MAE: 0.784 (0.190)


In [65]:
X_val.head(2)

,temp,conductivity,velocity
95,25.0,17.91,1509.900024
55,10.0,9.31,1424.900024


In [75]:
# GET THIS TESTED LATER WITH OUR OWN DATASET
# load dataset
# X, y = get_dataset()
n_inputs, n_outputs = X_norm.shape[1], y_train.shape[1]
# get model
model = get_model(n_inputs, n_outputs)
# fit the model on all data
model.fit(X_norm, y_train, verbose=0, epochs=100)
# make a prediction for new data
row = [-0.99859353,2.19284309,-0.42632569,-0.21043258,-1.13655612,-0.55671602,-0.63169045,-0.87625098,-0.99445578,-0.3677487]
newX = np.asarray(X_norm)
yhat = model.predict(newX)
print('Predicted: %s' % yhat[2])

Predicted: [2.0275056 1.5601187]


In [78]:
y_train

,nacl,aa
82,3.0,2.8
36,3.0,0.5
100,3.0,1.0
103,3.0,2.8
25,0.0,2.0
...,...,...
16,3.0,1.0
98,3.0,0.2
23,0.0,1.0
79,3.0,1.0


In [ ]:
# from sklearn.linear_model import LinearRegression
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.tree import DecisionTreeRegressor


# evaluate multioutput regression model with k-fold cross-validation
# from numpy import absolute
# from numpy import mean
# from numpy import std
# from sklearn.datasets import make_regression
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import RepeatedKFold
# # create datasets
# X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
# # define model
# model = DecisionTreeRegressor()
# # define the evaluation procedure
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# # evaluate the model and collect the scores
# n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# # force the scores to be positive
# n_scores = absolute(n_scores)
# # summarize performance
# print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# DIRECT MULTIOUTPUT REGRESSION

	
# example of making a prediction with the direct multioutput regression model
# from sklearn.datasets import make_regression
# from sklearn.multioutput import MultiOutputRegressor
# from sklearn.svm import LinearSVR
# # define dataset
# X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
# # define base model
# model = LinearSVR()
# # define the direct multioutput wrapper model
# wrapper = MultiOutputRegressor(model)
# # fit the model on the whole dataset
# wrapper.fit(X, y)
# # make a single prediction
# row = [0.21947749, 0.32948997, 0.81560036, 0.440956, -0.0606303, -0.29257894, -0.2820059, -0.00290545, 0.96402263, 0.04992249]
# yhat = wrapper.predict([row])
# # summarize the prediction
# print('Predicted: %s' % yhat[0])

# CHAIN MULTIOUTPUT REGRESSION
	
# # example of making a prediction with the chained multioutput regression model
# from sklearn.datasets import make_regression
# from sklearn.multioutput import RegressorChain
# from sklearn.svm import LinearSVR
# # define dataset
# X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
# # define base model
# model = LinearSVR()
# # define the chained multioutput wrapper model
# wrapper = RegressorChain(model)
# # fit the model on the whole dataset
# wrapper.fit(X, y)
# # make a single prediction
# row = [0.21947749, 0.32948997, 0.81560036, 0.440956, -0.0606303, -0.29257894, -0.2820059, -0.00290545, 0.96402263, 0.04992249]
# yhat = wrapper.predict([row])
# # summarize the prediction
# print('Predicted: %s' % yhat[0])